# 스태킹 앙상블

*   원본 데이터의 train, test가 존재
*원본 training data를 3개의 머신러닝 모델이 학습
*각 모델마다 X_test를 넣어서 예측 후 predict를 뽑아냄 (3개의 predict된 값)
*3개의 predict를 다시 학습 데이터로 사용
*최종 model을 하나 선정해 학습
*최종 평가

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%reload_ext autoreload
%autoreload 2
!pip install kaggler

     |████████████████████████████████| 822 kB 5.3 MB/s 
  Created wheel for kaggler: filename=Kaggler-0.9.13-cp37-cp37m-linux_x86_64.whl size=2963688 sha256=7644067afd0fc204c6e7234e96f88950cefe422091ef32b5a42a3271ad976530
  Stored in directory: /root/.cache/pip/wheels/64/38/23/298e2f6a012a8ae187cf100410865a2cdd434aed5a75a42cfe
  Created wheel for ml-metrics: filename=ml_metrics-0.1.4-py3-none-any.whl size=7845 sha256=60cd75597178c61562a78d1112096ab554475104face3772fed6b3553cf4dc70
  Stored in directory: /root/.cache/pip/wheels/56/41/5b/0c6d42b3604a5c823d8922564c4708f84962fa7f2f4facfa6d
Successfully built kaggler ml-metrics


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from pprint import pprint
from copy import copy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix

from kaggler.preprocessing import LabelEncoder
from kaggler.model import AutoLGB
import kaggler

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pd.set_option('max_columns', 100)

In [ ]:
cd '/content/drive/MyDrive/DCC/data'

/content/drive/MyDrive/DCC/data


In [ ]:
train = pd.read_csv('./train_n.csv')
test = pd.read_csv("./test_n.csv")
submission = pd.read_csv('./sample_submission.csv')

In [ ]:
algo_name = 'esb'
model_name = f'{algo_name}'

predict_val_file = f'{model_name}.val.txt'
predict_tst_file = f'{model_name}.tst.txt'
submission_file = f'{model_name}.sub.csv'

In [ ]:
n_est = 3000
seed = 42
n_fold = 5
n_class = 3

gbm_params={
    'max_depth':7,
    'n_estimators':200,
    'max_features':0.4,
    'learning_rate' : 0.2,
     'min_samples_split': 16,
     'min_samples_leaf': 3
}
lgb_params = {
     'n_estimators':n_est,
     'min_samples_split': 0.01,
     'num_leaves': 445, 
     'min_child_samples':30,
     'learning_rate': 0.01, 
     'max_bin':700, 
     'metric': 'multiclass',
     'feature_fraction': 0.4,
     'min_data_in_leaf':17,
     'bagging_fraction':0.9,
     'bagging_freq':8
     
}
rf_params = {
    'max_depth': 35,
    'min_samples_leaf': 2,
    'n_estimators': 3000,
    'min_samples_split':3,
    'random_state': seed
}
et_params = {
    'bootstrap' : True, 
    'criterion':'entropy', 
    'max_features':0.50, 
    'min_samples_leaf':2, 
    'min_samples_split':3, 
    'n_estimators' :1000
}
ada_params = {
    'n_estimators': 30, 
    'random_state':10, 
    'learning_rate':0.1
}


In [ ]:
base_models = {            
            'gbm':GradientBoostingClassifier(**gbm_params),
            'lgb': LGBMClassifier(**lgb_params),
            'rf':RandomForestClassifier(**rf_params),
            'et': ExtraTreesClassifier(**et_params),              
            'ada': AdaBoostClassifier(**ada_params)
}

In [ ]:
y = train['credit']
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)


In [ ]:
X = train.drop(['credit'],axis=1)
y = train['credit']
X_tst = test

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

p_dict = {}
p_tst_dict = {}
for name in base_models:
    print(f'Training {name}:')
    p = np.zeros((X.shape[0], n_class), dtype=float)
    p_tst = np.zeros((X_tst.shape[0], n_class), dtype=float)
    for i, (i_trn, i_val) in enumerate(cv.split(X, y)):
        clf = copy(base_models[name])
        if name=='lgb':
            clf.fit(X.iloc[i_trn], y[i_trn], eval_set=[(X.iloc[i_trn], y[i_trn]), (X.iloc[i_val], y[i_val])], early_stopping_rounds=30,
            verbose=1000)
        else:
            clf.fit(X.iloc[i_trn], y[i_trn])

        p[i_val] = clf.predict_proba(X.iloc[i_val])
        p_tst += clf.predict_proba(X_tst) / n_fold

    p_dict[name] = p
    p_tst_dict[name] = p_tst
    pd.DataFrame(p).to_csv(f'./제출/{name}.csv')
    pd.DataFrame(p_tst).to_csv(f'./제출/{name}_tst.csv')

    print(f'\tCV Log Loss: {log_loss(y, p):.6f}')
    

Training gbm:
	CV Log Loss: 0.739845
Training lgb:
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[603]	valid_0's multi_logloss: 0.358664	valid_1's multi_logloss: 0.684908
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[548]	valid_0's multi_logloss: 0.375603	valid_1's multi_logloss: 0.700605
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[539]	valid_0's multi_logloss: 0.377416	valid_1's multi_logloss: 0.704329
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[572]	valid_0's multi_logloss: 0.367077	valid_1's multi_logloss: 0.697247
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[528]	valid_0's multi_logloss: 0.382358	valid_1's multi_logloss: 0.704335
	CV Log Loss: 0.698284
Training rf:
	CV Log Loss: 0.699821
Training et:
	CV Log Loss: 0.721522

In [ ]:
p_tst_dict

{'ada': array([[0.25216106, 0.29859403, 0.44924491],
        [0.25216106, 0.29859403, 0.44924491],
        [0.25261785, 0.29855025, 0.4488319 ],
        ...,
        [0.25329142, 0.29758703, 0.44912155],
        [0.25216106, 0.29859403, 0.44924491],
        [0.2503173 , 0.30365029, 0.4460324 ]]),
 'et': array([[0.07295537, 0.1875081 , 0.73953653],
        [0.32034789, 0.23026088, 0.44939123],
        [0.06186633, 0.12392712, 0.81420655],
        ...,
        [0.04625515, 0.11910613, 0.83463871],
        [0.24470173, 0.42689938, 0.32839889],
        [0.09412563, 0.26481957, 0.64105479]]),
 'gbm': array([[0.03064637, 0.13519151, 0.83416212],
        [0.29312117, 0.25550325, 0.45137558],
        [0.02679391, 0.06721683, 0.90598926],
        ...,
        [0.01150314, 0.16816494, 0.82033192],
        [0.13323202, 0.3576289 , 0.50913908],
        [0.03616929, 0.20785665, 0.75597406]]),
 'lgb': array([[0.08826717, 0.16022322, 0.75150961],
        [0.29940161, 0.2590812 , 0.44151719],
        

In [ ]:
# import pickle
# # save data
# with open('p_dict.pickle','wb') as fw:
#     pickle.dump(p_dict, fw)
# with open('p_tst_dict.pickle','wb') as fw:
#     pickle.dump(p_tst_dict, fw)

In [ ]:
# import pickle
# with open('p_dict.pickle', 'rb') as fr:
#     p_dict = pickle.load(fr)
# with open('p_tst_dict.pickle', 'rb') as fr:
#     p_tst_dict = pickle.load(fr)

In [ ]:
p_tst_dict

{'ada': array([[0.25216106, 0.29859403, 0.44924491],
        [0.25216106, 0.29859403, 0.44924491],
        [0.25261785, 0.29855025, 0.4488319 ],
        ...,
        [0.25329142, 0.29758703, 0.44912155],
        [0.25216106, 0.29859403, 0.44924491],
        [0.2503173 , 0.30365029, 0.4460324 ]]),
 'et': array([[0.07295537, 0.1875081 , 0.73953653],
        [0.32034789, 0.23026088, 0.44939123],
        [0.06186633, 0.12392712, 0.81420655],
        ...,
        [0.04625515, 0.11910613, 0.83463871],
        [0.24470173, 0.42689938, 0.32839889],
        [0.09412563, 0.26481957, 0.64105479]]),
 'gbm': array([[0.03064637, 0.13519151, 0.83416212],
        [0.29312117, 0.25550325, 0.45137558],
        [0.02679391, 0.06721683, 0.90598926],
        ...,
        [0.01150314, 0.16816494, 0.82033192],
        [0.13323202, 0.3576289 , 0.50913908],
        [0.03616929, 0.20785665, 0.75597406]]),
 'lgb': array([[0.08826717, 0.16022322, 0.75150961],
        [0.29940161, 0.2590812 , 0.44151719],
        

In [ ]:
submission.iloc[:,1:] = p_tst_dict['lgb']
submission.to_csv('./submissionfile_lgb.csv', index=False)
# 점수 0.68956 lgb단일

* AutoLGB를 사용하여 최적 파라미터를 구한다음, 다음 폴드부터는 동일한 파라미터를 사용

In [ ]:
X = pd.DataFrame(np.hstack([x for _, x in p_dict.items()]))
X_tst = pd.DataFrame(np.hstack([x for _, x in p_tst_dict.items()]))

p = np.zeros((X.shape[0], n_class), dtype=float)
p_tst = np.zeros((X_tst.shape[0], n_class), dtype=float)
for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y)):
    print(i_cv)
    if i_cv == 0:
        clf = AutoLGB(objective='multiclass', metric='multi_logloss', params={'num_class': n_class}, 
                        feature_selection=False, n_est=10000)
        clf.tune(X.iloc[i_trn], y[i_trn])
        n_best = clf.n_best
        features = clf.features
        params = clf.params
        #  params['learning_rate'] = 
        print(f'best iteration: {n_best}')
        print(f'selected features ({len(features)}): {features}')        
        pprint(params)
        clf.fit(X.iloc[i_trn], y[i_trn])
    else:
        train_data = lgb.Dataset(X[features].iloc[i_trn], label=y[i_trn])
        clf = lgb.train(params, train_data, n_best, verbose_eval=100)
    p[i_val] = clf.predict(X[features].iloc[i_val])
    p_tst += clf.predict(X_tst[features]) / n_fold


    print(f'CV Log Loss: {log_loss(y, p):.6f}')
np.savetxt(predict_val_file, p, fmt='%.6f')
np.savetxt(predict_tst_file, p_tst, fmt='%.6f')

0
100%|██████████| 100/100 [01:49<00:00,  1.09s/it, best loss: 0.6865115856397561]
best iteration: 286
selected features (15): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'bagging_fraction': 0.7000000000000001,
 'bagging_freq': 1,
 'boosting': 'gbdt',
 'feature_fraction': 0.5,
 'feature_pre_filter': False,
 'lambda_l1': 0.1,
 'lambda_l2': 0.1,
 'learning_rate': 0.018653458479360056,
 'max_depth': 4,
 'metric': 'multi_logloss',
 'min_child_samples': 25,
 'num_class': 3,
 'num_leaves': 255,
 'num_threads': -1,
 'objective': 'multiclass',
 'seed': 42,
 'verbosity': -1}
CV Log Loss: 1.012839
1
CV Log Loss: 0.931369
2
CV Log Loss: 0.848808
3
CV Log Loss: 0.765796
4
CV Log Loss: 0.683448


In [ ]:
submission.iloc[:,1:]=p_tst
submission.to_csv('./submissionfile_stacking.csv',index=False)
# 점수 0.680383931 